### Bag of Words(BoW)
- Bag of Words란 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법
- BoW는 각 단어가 등장한 횟수를 수치화하는 텍스트 표현 방법이기 때문에, 주로 어떤 단어가 얼마나 등장했는지를 기준으로 문서가 어떤 성격의 문서인지를 판단하는 작업에 쓰임.
- 분류 문제나 여러 문서 간의 유사도를 구하는 문제에 주로 쓰임.
- 가령, '달리기', '체력', '근력'과 같은 단어가 자주 등장하면 해당 문서를 체육 관련 문서로 분류할 수 있을 것임.
- '미분', '방정식', '부등식'과 같은 단어가 자주 등장한다면 수학 관련 문서로 분류할 수 있음.
- BoW를 만드는 과정<br>
(1) 우선, 각 단어에 고유한 정수 인덱스를 부여합니다.<br>
(2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듭니다.

In [1]:
from konlpy.tag import Okt # Twitter 자연어 처리 package 
import re  # 정규 표현식 package 
okt=Okt()  

# "."을 찾아서 ""로 변경, 정규 표현식을 통해 온점을 제거하는 정제 작업입니다. 
token=re.sub("(\.)","","이번주 월요일은 휴강 다음주 월요일도 휴강 2틀간 휴강.") 
# Okt 형태소(더이상 분리 할 수 없는 의미를 가지고 있는 단어) 분석기를 통해
# 토큰화(분리) 작업을 수행
token=okt.morphs(token)  

print(token)

['이번', '주', '월요일', '은', '휴강', '다음주', '월요일', '도', '휴강', '2', '틀', '간', '휴강']


In [2]:
word2index={} # 키:값 
bow=[]        # 배열 
for voca in token: # '이번', '주', '월요일'...
    if voca not in word2index.keys():   # 새로운 형태소가 key에 없다면
        word2index[voca]=len(word2index) # '이번': 0, '주': 1, '월요일': 2... 
        # token을 읽으면서, word2index에 없는 (not in) 단어는 새로 추가하고,
        # 이미 있는 단어는 넘깁니다.   
        bow.insert(len(word2index)-1,1) # bow.insert(0, 1)
        # 단어를 최초 등록중임으로 BoW 전체에 전부 기본값 1을 넣어줍니다. 
        # 단어의 개수 만큼 1이 list에 추가됨.
    else: # word2index에 존재한다면
        # 재등장하는 단어의 값을 가져와서 bow list의 인덱스로 사용
        index=word2index.get(voca) 
        bow[index]=bow[index]+1 # 동일한 단어 등장시 카운트 1 증가

In [3]:
print(word2index)

{'이번': 0, '주': 1, '월요일': 2, '은': 3, '휴강': 4, '다음주': 5, '도': 6, '2': 7, '틀': 8, '간': 9}


In [4]:
print(bow)

[1, 1, 2, 1, 3, 1, 1, 1, 1, 1]


In [5]:
# 함수로 제작
def bow_process(src):
    okt=Okt()  
    token=re.sub("[\.%]","",src) # ., % 제거
    token=okt.morphs(token) 

    print(token)

    word2index={}  
    bow=[]  
    for voca in token: 
        if voca not in word2index.keys(): 
            word2index[voca]=len(word2index)
            bow.insert(len(word2index)-1,1)
        else:
            index=word2index.get(voca)
            bow[index]=bow[index]+1
    return word2index, bow

In [6]:
word2index, bow = bow_process('월요일 화요일 월요일 수요일 월요일')
print(word2index)
print(bow)

['월요일', '화요일', '월요일', '수요일', '월요일']
{'월요일': 0, '화요일': 1, '수요일': 2}
[3, 1, 1]


In [7]:
word2index, bow = bow_process('이재명 더불어민주당 대선후보가 윤석열 국민의힘 대선후보와 오차범위인 1%포인트 격차로 앞서며 초접전을 벌이는 것으로 나타났다.')
print(word2index)
print(bow)

['이재명', '더불어', '민주당', '대선', '후보', '가', '윤석열', '국민', '의', '힘', '대선', '후보', '와', '오차', '범위', '인', '1', '포인트', '격차', '로', '앞서며', '초', '접전', '을', '벌이는', '것', '으로', '나타났다']
{'이재명': 0, '더불어': 1, '민주당': 2, '대선': 3, '후보': 4, '가': 5, '윤석열': 6, '국민': 7, '의': 8, '힘': 9, '와': 10, '오차': 11, '범위': 12, '인': 13, '1': 14, '포인트': 15, '격차': 16, '로': 17, '앞서며': 18, '초': 19, '접전': 20, '을': 21, '벌이는': 22, '것': 23, '으로': 24, '나타났다': 25}
[1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### CountVectorizer 클래스로 BoW 만들기
- 공백을 구분값으로하여 빈도수를 산출한다.
- 2개 이상의 단어를 대상으로 빈도수를 산출함.
- 문서를 토큰 리스트로 변환한다.
- 각 문서에서 토큰의 출현 빈도를 센다.
- 각 문서를 BOW 인코딩 벡터로 변환한다.
- min_df=0: 빈도수가 설정된 값보다 작으면 사용하지 않음,0은 모든 빈도 사용, 문서를 구성하는 문장 기준 예) 2개의 문장에서 2번 발생한 단어 필터링시 선언: min_df=2 
- ngram_range=(1, 1): 빈도수를 계산할 단어를 조합할 문자의 수
- CountVectorizer는 단지 띄어쓰기만을 기준으로 단어를 자르는 낮은 수준의 토큰화를 진행
- 영어의 경우 띄어쓰기만으로 토큰화가 수행되기 때문에 문제가 없음.
- 한국어에 CountVectorizer를 적용하면, 조사 등의 이유로 제대로 BoW가 만들어지지 않을 수 있음
- CountVectorizer는 띄어쓰기를 기준으로 분리한 뒤에 '물가상승률과'와 '물가상승률은'으로 조사를 포함해서<br>
  하나의 단어로 판단하기 때문에 서로 다른 두 단어로 인식합니다.<br>
  그리고 '물가상승률과'와 '물가상승률은'이 각자 다른 인덱스에서 1이라는 빈도의 값을 갖게 됩니다.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# 말뭉치: 처리하려는 모든 문장/단어들의 집합
corpus = ['you know I want your love. because I love you.']

vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray()) # 단어의 빈도 수 산출
# 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다. 알파벳 순서
print(vector.vocabulary_) 

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


### 불용어를 제거한 BoW 만들기
- 불용어를 제거하는 일은 자연어 처리의 정확도를 높이기 위해서 선택할 수 있는 전처리 기법임.

In [9]:
# (1) 사용자가 직접 정의한 불용어 사용
from sklearn.feature_extraction.text import CountVectorizer

text=["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print(vect.fit_transform(text).toarray())  # 단어의 빈도 수 산출
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [11]:
# (2) CountVectorizer에서 제공하는 자체 불용어 사용
from sklearn.feature_extraction.text import CountVectorizer

text=["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english") # 불용어 대상을 영어로 지정
print(vect.fit_transform(text).toarray())    # 단어의 빈도 수 산출
print(vect.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


In [12]:
# (3) NLTK에서 지원하는 불용어 사용
import nltk 

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soldesk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

text=["Family is not an important thing. It's everything."]
# 불용어 지원 언어 목록 확인 폴더: C:\Users\stu\AppData\Roaming\nltk_data\corpora\stopwords
sw = stopwords.words("english") # 영어 관련 불용어 자동 제공됨.
vect = CountVectorizer(stop_words =sw)
print(vect.fit_transform(text).toarray()) # 단어의 빈도 수 산출
print(vect.vocabulary_)

[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


In [14]:
text=["사과에 딸기 그리고 딸기와 포도 포도"]
vect = CountVectorizer()
print(vect.fit_transform(text).toarray())  # 단어의 빈도 수 산출
print(vect.vocabulary_)

[[1 1 1 1 2]]
{'사과에': 3, '딸기': 1, '그리고': 0, '딸기와': 2, '포도': 4}


In [16]:
text = ['사과 딸기', '딸기 바나나', '수박 수박'] # 문장 3개
# min_df=0: 모든 빈도 포함, ngram_range=(1, 1): 단어를 1개씩 분리
vect = CountVectorizer(min_df=0, ngram_range=(1, 1))
fruit_vector = vect.fit_transform(text) # 공백으로 값을 구분하여 빈도수 산출
print(fruit_vector.shape) 
print(fruit_vector.toarray())
print(vect.vocabulary_)
#              딸기 바나나 사과 수박 <-- 단어 사전
# 사과 딸기    1    0      1    0 
# 딸기 바나나  1    1      0    0 
# 수박 수박    0    0      0    2

(3, 4)
[[1 0 1 0]
 [1 1 0 0]
 [0 0 0 2]]
{'사과': 2, '딸기': 0, '바나나': 1, '수박': 3}


In [18]:
text = ['사과 딸기', '딸기 바나나', '딸기', '수박 수박']
# min_df=1: 빈도가 1이상 포함, ngram_range=(1, 1): 단어를 1개 분리
vect = CountVectorizer(min_df=1, ngram_range=(1, 1))
fruit_vector = vect.fit_transform(text) # 공백으로 값을 구분하여 빈도수 산출
print(fruit_vector.shape) 
print(fruit_vector.toarray())
print(vect.vocabulary_)
#              딸기 바나나 사과 수박 
# 사과 딸기    1    0      1    0 
# 딸기 바나나  1    1      0    0 
# 딸기         1    0      0    0 
# 수박 수박    0    0      0    2 <- 포함안됨, 하나의 문장에서만 출현

(4, 4)
[[1 0 1 0]
 [1 1 0 0]
 [1 0 0 0]
 [0 0 0 2]]
{'사과': 2, '딸기': 0, '바나나': 1, '수박': 3}


In [24]:
# 하나의 글자로 구성된 '배'는 제외됨.
text = ['사과 딸기', '딸기 바나나', '배', '수박 수박']
# min_df=0: 빈도가 0이상 포함, ngram_range=(1, 1): 단어를 1개 분리
vect = CountVectorizer(min_df=0, ngram_range=(1, 1))
fruit_vector = vect.fit_transform(text) # 공백으로 값을 구분하여 빈도수 산출
print(fruit_vector.shape) 
print(fruit_vector.toarray())
print(vect.vocabulary_)
#              딸기 바나나 사과 수박 
# 사과 딸기    1    0      1    0 
# 딸기 바나나  1    1      0    0 
# 배           0    0      0    0 
# 수박 수박    0    0      0    2 <- 포함안됨, 하나의 문장에서만 출현

(4, 4)
[[1 0 1 0]
 [1 1 0 0]
 [0 0 0 0]
 [0 0 0 2]]
{'사과': 2, '딸기': 0, '바나나': 1, '수박': 3}


In [25]:
# min_df=2: 빈도가 2이상 포함, ngram_range=(1, 1): 단어를 1개 분리
text = ['사과 딸기', '딸기 바나나', '배', '수박 수박']
vect = CountVectorizer(min_df=2, ngram_range=(1, 1))
fruit_vector = vect.fit_transform(text) # 공백으로 값을 구분하여 빈도수 산출
print(fruit_vector.shape) 
print(fruit_vector.toarray()) # 공백으로 값을 구분하여 빈도수 출력
print(vect.vocabulary_)
#              딸기 <- 4개의 문장에 등장한 빈도수가 2번 이상인 단어(토큰)
# 사과 딸기    1  
# 딸기 바나나  1  
# 배           0 
# 수박 수박    0 <- 하나의 문장에서 빈도수는 2번이나 문장 기준 전체 문서에서는 1번 등장

(4, 1)
[[1]
 [1]
 [0]
 [0]]
{'딸기': 0}


In [57]:
# 단어를 1~2개 분리
text = ['사과 딸기', '딸기 바나나', '수박', '수박 수박 수박']
# min_df=2: 빈도가 2이상 포함, ngram_range=(1, 2): 단어를 1~2개 분리
vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
fruit_vector = vect.fit_transform(text) # 공백으로 값을 구분하여 빈도수 산출
print(fruit_vector.shape) 
print(fruit_vector.toarray())
print(vect.vocabulary_)
#             '딸기' '딸기 바나나' '바나나' '사과' '사과 딸기' '수박' '수박 수박'
# 사과 딸기    1                             1
# 딸기 바나나  1      1             1
# 수박                                                          1
# 수박 수박 수박                                                3      2

(4, 7)
[[1 0 0 1 1 0 0]
 [1 1 1 0 0 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 3 2]]
{'사과': 3, '딸기': 0, '사과 딸기': 4, '바나나': 2, '딸기 바나나': 1, '수박': 5, '수박 수박': 6}
